In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)

In [7]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002529C65D930>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002529C65DCC0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="hello How are you?")
]

response = model.invoke(messages)

In [11]:
response

AIMessage(content="Hello - Bonjour \n\nHow are you? - Comment allez-vous ? \n\n\nLet me know if you'd like to translate something else!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 21, 'total_tokens': 54, 'completion_time': 0.06, 'prompt_time': 0.002139347, 'queue_time': 0.255983912, 'total_time': 0.062139347}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--261c6f46-7118-4821-ac48-7dc59b3e6406-0', usage_metadata={'input_tokens': 21, 'output_tokens': 33, 'total_tokens': 54})

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(response)

"Hello - Bonjour \n\nHow are you? - Comment allez-vous ? \n\n\nLet me know if you'd like to translate something else!\n"

In [13]:
### using LCEL  - chain the components

chain = model | parser
chain.invoke(messages)

"Here's the translation:\n\n* **Hello:** Bonjour\n* **How are you?:** Comment allez-vous? (formal) or Comment vas-tu? (informal) \n\n\nLet me know if you'd like to translate anything else! \n"

In [14]:
### Prompt Templates

from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {langauge}"

prompt = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])

In [15]:
prompt

ChatPromptTemplate(input_variables=['langauge', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['langauge'], input_types={}, partial_variables={}, template='Translate the following into {langauge}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [17]:
chain = prompt | model | parser

response = chain.invoke({
    "langauge":"French",
    "text":"Hello can you help me?"
})
response

"Bonjour, pouvez-vous m'aider ? \n"